## Import and process the public dataset


### 1. Download and extract original files
First, let's download the IDD microdata. From http://inep.gov.br/microdados, under the section heading "Indicador da Diferença entre os Desempenhos Observado e Esperado (IDD)"

In [1]:
import os,sys,zipfile
import requests
original = 'data/original'

URLs = ["http://download.inep.gov.br/microdados/microdados_idd_2014.zip",
        "http://download.inep.gov.br/microdados/microdados_idd_2015.zip",
        "http://download.inep.gov.br/microdados/microdados_idd_2016.zip",
        "http://download.inep.gov.br/microdados/microdados_idd_2017.zip"]
for url in URLs:
    fn = url.split('/')[-1]
    if os.path.isfile(os.path.join(original,fn)):
        print("skipping, {} already download".format(fn))
    else:
        r = requests.get(url, allow_redirects=True)
        open(os.path.join(original,fn), 'wb').write(r.content)
    ano = fn.split('_')[-1][:4]
    if not os.path.isdir(os.path.join(original,ano)):
        zipf = zipfile.ZipFile(os.path.join(original,fn), 'r')
        zipf.extractall(os.path.join(original,ano))
        zipf.close()

skipping, microdados_idd_2014.zip already download
skipping, microdados_idd_2015.zip already download
skipping, microdados_idd_2016.zip already download
skipping, microdados_idd_2017.zip already download


Second, the oficial results for the indicators per course, from [here](http://portal.inep.gov.br/web/guest/indicadores-de-qualidade). Ideally, we'd use the oficial spreadsheets with [the course IDDs](http://portal.inep.gov.br/web/guest/educacao-superior/indicadores-de-qualidade/indicador-de-diferenca-entre-os-desempenhos-observado-e-esperado-idd), but these data are currently only available for 2016 and 20017. Therefore, for 2014 and 2015 we use spreadsheets for the "CPC" (Conceito Preliminar do Curso) indicator, from [here](http://portal.inep.gov.br/web/guest/conceito-preliminar-de-curso-cpc-). 

However, it must be noted that before 2015, these spreadsheets only contain the IDD results aggregated by the so-called "Unidade de observação" (observed unit), which are all the courses within an institution (university or college) within one municipality. Some courses have more than one "version" within one institution and municipality (for example night-time and day-time courses), and the CPC and its inputs like the IDD were not reported separately before 2015. 

In [2]:
cpcURLs = ["http://download.inep.gov.br/educacao_superior/enade/planilhas/2012/cpc_2012_site_2014_03_14.xls",
           "http://download.inep.gov.br/educacao_superior/enade/planilhas/2013/cpc2013_atualizado_em_27112017.xls",
           "http://download.inep.gov.br/educacao_superior/enade/planilhas/2014/cpc2014_atualizado_em_04122017.xlsx",
           "http://download.inep.gov.br/educacao_superior/indicadores/legislacao/2017/cpc_2015_portal_atualizado_03_10_2017.xls",
           "http://download.inep.gov.br/educacao_superior/indicadores/legislacao/2017/Resultado_CPC_2016_portal_23_02_2018.xls",
           "http://download.inep.gov.br/educacao_superior/igc_cpc/2018/resultado_cpc_2017.xlsx"
        ]

iddURLs = ["http://download.inep.gov.br/educacao_superior/indicadores/legislacao/2017/resultado_IDD2016_portal_06_09_2017.xlsx",
           "http://download.inep.gov.br/educacao_superior/indicadores/legislacao/2018/resultados_IDD_2017.xlsx"]

for url in cpcURLs + iddURLs:
    fn = url.split('/')[-1]
    if os.path.isfile(os.path.join(original,fn)):
        print("skipping, {} already download".format(fn))
    else:
        r = requests.get(url, allow_redirects=True)
        open(os.path.join(original,fn), 'wb').write(r.content)

skipping, cpc_2012_site_2014_03_14.xls already download
skipping, cpc2013_atualizado_em_27112017.xls already download
skipping, cpc2014_atualizado_em_04122017.xlsx already download
skipping, cpc_2015_portal_atualizado_03_10_2017.xls already download
skipping, Resultado_CPC_2016_portal_23_02_2018.xls already download
skipping, resultado_cpc_2017.xlsx already download
skipping, resultado_IDD2016_portal_06_09_2017.xlsx already download
skipping, resultados_IDD_2017.xlsx already download


### 2. Process the orginal data
We'll put processed, intermediate data files into data/processed as csv files.

First, the **microdata**. The download and unzip above should have created the following comma separated microdata, which we will transform into files with standardized column names. For now, we'll only use the microdata of 2014 and 2017 (which are the only years for which the same areas were evaluated with the new "ENEM" methodology).

In [25]:
import pandas as pd
processed = 'data/processed'
files = {2014:'data/original/2014/Dados/microdados_idd_2014.csv',
         2015:'data/original/2015/microdados_idd_2015/2.Dados/microdados_idd_2015.csv',
         2016:'data/original/2016/2.Dados/microdados_idd_2016.csv',
         2017:'data/original/2017/3.DADOS/MICRODADOS_IDD_2017.txt'}

def processes_idd_microdata(df):
    df = df[['ano','area','ies','catacad','orgacad','curso','municipio','ingresso','nota','cn','ch','lc','mt']]
    ano = df['ano'][0]
    df.to_csv(os.path.join(processed,"idd-microdata-{}.csv".format(ano)),index=False)
    return df

# 2014
oldcols = ['nu_ano','co_grupo','co_ies','co_catad','co_orgac','co_munic_curso','co_curso','ano_in_grad','tp_inscricao','tp_pres','nt_ger','enem_nt_cn','enem_nt_ch','enem_nt_lc','enem_nt_mt']
df = pd.read_table(files[2014],sep=';',usecols=oldcols)
df.columns = ['ano','area','ies','catacad','orgacad','municipio','curso','ingresso','tp_inscricao','tp_pres','nota','cn','ch','lc','mt']
df = df[df['tp_pres'] == 555] # presente com resultado válido
df = df[df['tp_inscricao'] == 0] # Concluinte
df = processes_idd_microdata(df)

#2017
oldcols = ['NU_ANO','CO_IES','CO_CATEGAD','CO_ORGACAD','CO_GRUPO','CO_CURSO','CO_MUNIC_CURSO','ANO_IN_GRAD','NT_GER','VL_NOTA_ENEM_CN','VL_NOTA_ENEM_CH','VL_NOTA_ENEM_LC','VL_NOTA_ENEM_MT']
df = pd.read_table(files[2017],sep=';',usecols=oldcols)
df.columns = ['ano','ies','catacad','orgacad','area','curso','municipio','ingresso','nota','cn','ch','lc','mt']
df = processes_idd_microdata(df)

Second, the **spreadsheets** with the results INEP published. Let's transform the ridiculous spreadsheets into nice csv files. We have the IDD spreadsheets for 2016 and 2017:

In [11]:
idd17 = pd.read_excel('data/original/resultados_IDD_2017.xlsx',usecols=[1,8,15,17])
idd17.columns = ['area','curso','N','idd']
idd17.curso = idd17.curso.astype('str')
idd17['ano'] = 2017
idd17.to_csv(os.path.join(processed,"idd-pub-2017.csv"),index=False)

In [12]:
idd16 = pd.read_excel('data/original/resultado_IDD2016_portal_06_09_2017.xlsx',usecols=[6,7,15,17])
idd16.columns = ['curso','area','N','idd']
idd16 = idd16[['area','curso','N','idd']]
idd16.curso = idd16.curso.astype('str')
idd16['ano'] = 2016
idd16.to_csv(os.path.join(processed,"idd-pub-2016.csv"),index=False)

But the same information is in the **CPC spreadsheets**. Unfortunately, as noted above, in the CPC spreadsheets the "código do curso" is only identified after 2015: "O CPC, assim como o Conceito Enade, também passou a ser calculado, a partir da edição de 2015, por curso de graduação, identificado pelo código do curso constante no Sistema e-MEC" (see [here](http://portal.inep.gov.br/conceito-preliminar-de-curso-cpc-))

In [164]:
def xls2csv(ano,fn,cols,colnames):
    df = pd.read_excel(fn,usecols=cols,na_values=['-','.'])
    df.columns = colnames
    if ano in [2015,2016]:
        df = df[['area','ies','municipio','curso','N','fracN','idd','cpc','cpcf']]
    if ano == 2012:
        df = df[['area','ies','municipio','Ncursos','N','idd','cpc','cpcf']]
    df['ano'] = ano
    return df

In [124]:
cols = pd.read_excel(cpcfiles[2016]).columns

In [131]:
cols[1]

'Código da IES'

In [162]:
cpcfiles = {2017:'data/original/resultado_cpc_2017.xlsx',
            2016: 'data/original/Resultado_CPC_2016_portal_23_02_2018.xls',
            2015: 'data/original/cpc_2015_portal_atualizado_03_10_2017.xls',
            2014: 'data/original/cpc2014_atualizado_em_04122017.xlsx',
            2013: 'data/original/cpc2013_atualizado_em_27112017.xls',
            2012: 'data/original/cpc_2012_site_2014_03_14.xls'}
cpccols = {2017: [[1,3,8,10,13,14,17,18,19,20,35,36],['area','ies','curso','municipio','Nci','Ncp','enade','N','fracN','idd','cpc','cpcf']], #N = conluintes com nota no enem
           2016: [[1,6,7,10,18,19,20,35,36],['ies','curso','area','municipio','N','fracN','idd','cpc','cpcf']], #idd = Nota Bruta - IDD
           2015: [[1,6,7,10,18,19,20,35,36], ['ies','curso','area','municipio','N','fracN','idd','cpc','cpcf']],
           2014: [[1,3,8,12,13,14,18,25,26,27,37,38], ['area','ies','municipio','Ncursos','Nci','Ncp','enade','N','fracN','idd','cpc','cpcf']],
           2013: [[1,3,8,12,14,29,37,38], ['area','ies','municipio','Ncursos','N','idd','cpc','cpcf']],
           2012: [[1,3,8,9,13,24,36,37], ['area','ies','Ncursos','municipio','N','idd','cpc','cpcf']]}

In [163]:
for ano in range(2012,2018):
    df = xls2csv(ano,cpcfiles[ano],cols=cpccols[ano][0],colnames = cpccols[ano][1])
    df.to_csv(os.path.join(processed,"cpc-pub-{}.csv".format(ano)),index=False)

In [159]:
ano = 2014
df = pd.read_csv('data/processed/cpc-pub-{}.csv'.format(ano))
df.dtypes

area           int64
ies            int64
municipio      int64
Ncursos        int64
N              int64
fracN        float64
idd          float64
cpc          float64
cpcf          object
ano            int64
dtype: object

In [155]:
df = pd.read_excel(cpcfiles[2012],usecols=cpccols[2012][0],na_values=['-','.'])
df.sample(5)

,Código da área,Código da IES,Número de cursos da unidade,Código do município do curso,Número concluintes participantes no Enade,Nota bruta do IDD,CPC (contínuo),CPC (faixa)
7055,804,4141,1,3554102,58,0.025767,3.149177,4
4994,84,322,1,3547304,12,3.550726,3.120806,4
2153,2,1064,1,3205309,91,3.144308,3.297670,4
3015,18,254,1,3550308,0,NaN,NaN,SC
1199,1,2334,1,3304524,25,-0.893397,2.264385,3


In [141]:
df.dtypes

Código da Área                                                int64
Código da IES                                                 int64
Código do Município                                           int64
Número de cursos da unidade                                   int64
Concluintes Participantes com nota no Enem                    int64
Percentual de Concluintes participantes com nota no Enem    float64
Nota Bruta - IDD                                            float64
CPC Contínuo                                                float64
CPC Faixa                                                    object
dtype: object